# HKJC Pre-Processor

Notebook for pre-processing the exctracted data from https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?

In [107]:
# load libraries
import pandas as pd
import numpy as np

# load page data
pages = pd.read_csv('pages.csv').iloc[:,1:]
horses = pd.read_csv('horses.csv').iloc[:,1:]

# define functions
def to_float(x):
    x = x.replace('-','')
    if len(x) == 0: return ''
    try: return float(x)
    except: return float(x[2:]) + int(x[0])*60

def length_to_float(x):
    x = x.replace('-', ' ').split()
    if len(x) == 1: 
        try: return float(x[0])
        except: return 0
    elif len(x) == 2:
        two = x[1].replace('/',' ').split()
        two = int(two[0])/int(two[1])
        return int(x[0]) + float(two)
    else: return 0
    
def isolate_bet_info(lines, target):
    temp, j, found = [], 0, False
    for line in lines:
        words = line.split()
        if target not in words:
            temp += words
            j += 1
        else: 
            found = True
            break
    if found: return temp, list(lines[j:])
    else: return temp, lines

### Race Data

In [112]:
# define extraction data structures
races = pd.DataFrame()

# process data in race_tab
for i in range(0,pages.shape[0])[:]:
    
    # define temp structure and quick values
    page = pd.DataFrame(np.zeros(1))
    page['date'] = pages.loc[i,'date']
    page['race'] = pages.loc[i,'race']
    text = pages.loc[i,'race_tab']
    
    # further extract by line
    text = text.splitlines()
    #print(text)
    line1 = text[0].split()
    line2 = text[1].split()
    line3 = text[2].split()
    line4 = text[3].split()
    line5 = pd.Series(text[4].split()[3:]).apply(float).tolist()
    
    # isolate data by line 1
    page['number'] = int(line1[-1][1:-1])
    
    # isolate data by line 2
    rating, going, distance, g, d , r= '', '', '', False, 0, 0
    for l in line2: # isolate going
        if l == ':': g = True
        elif g == True: going += l + ' '
    for l in line2: # isolate distance
        if l == '-': d += 1
        elif d == 1: 
            distance = int(l[:-1])
            break
    for l in line2: # isolate rating
        if l == '-': r += 1
        elif r == 2: 
            rating = l
            break
    page['class'] = line2[0] + ' ' + line2[1]
    page['distance'] = distance
    page['ratings'] = rating
    page['going'] = going
    
    # isolate data by line 3
    handicap = line3[0]
    for word in line3[1:-7]: handicap += ' ' + word

    page['handicap'] = handicap
    page['surface'] = line3[-4]
    page['surface_type'] = line3[-2][1:-1]
    
    # adj time data to floats for line 4
    page['pool'] = int(line4[1].replace(',',''))
    line4 = pd.Series(text[3].split()[4:]).apply(lambda x: x[1:-1]).apply(to_float).tolist()
    
    # extend line 4 & 5 to match 6 sections
    n = 6 - len(line4)
    for z in range(n):
        line4 += ['']
        line5 += ['']
    
    # isolate data by line 4
    page['time1'] = line4[-6]
    page['time2'] = line4[-5]
    page['time3'] = line4[-4]
    page['time4'] = line4[-3]
    page['time5'] = line4[-2]
    page['time6'] = line4[-1]
    
    # isolate data by line 5
    page['sec1'] = line5[-6]
    page['sec2'] = line5[-5]
    page['sec3'] = line5[-4]
    page['sec4'] = line5[-3]
    page['sec5'] = line5[-2]
    page['sec6'] = line5[-1]
    
    print(i, 'race info')
    
    # isolate dividend text
    text = pages.loc[i,'dividend_tab']
    
    # further extract lines to win & place data
    lines = text.splitlines()[2:]
    win, lines = isolate_bet_info(lines, 'PLACE')
    place, lines = isolate_bet_info(lines, 'QUINELLA')
    quinella, lines = isolate_bet_info(lines, 'PLACE')
    quinella_place, lines = isolate_bet_info(lines, 'PICK')
    pick31, lines = isolate_bet_info(lines, 'FORECAST')
    forecast, lines = isolate_bet_info(lines, 'TIERCE')
    tierce, lines = isolate_bet_info(lines, 'TRIO')
    trio, lines = isolate_bet_info(lines, 'FIRST')
    first4, lines = isolate_bet_info(lines, 'QUARTET')
    quartet, lines = isolate_bet_info(lines, 'Racing')
    
    # check place combos/div
    try: place_combination_3, place_div_3 = int(place[5]), float(place[6].replace(',',''))
    except: place_combination_3, place_div_3 = 0, 0
    try: 
        quinella_place_31, quinella_place_32 = int(quinella_place[6].split(',')[0]), int(quinella_place[6].split(',')[1])
        quinella_place_div_3 = float(quinella_place[7].replace(',',''))
    except: quinella_place_31, quinella_place_32, quinella_place_div_3 = 0, 0, 0
        
    # isolate easy lines
    page['win_combination'] = int(win[1])
    page['win_div'] = float(win[2].replace(',',''))
    page['place_combination_1'] = int(place[1])
    page['place_combination_2'] = int(place[3])
    page['place_combination_3'] = place_combination_3
    page['place_div_1'] = float(place[2].replace(',',''))
    page['place_div_2'] = float(place[4].replace(',',''))
    page['place_div_3'] = place_div_3
    
    # isolate quinella lines
    try:
        page['quinella_1'] = int(quinella[1].split(',')[0])
        page['quinella_2'] = int(quinella[1].split(',')[1])
        page['quinella_div'] = float(quinella[2].replace(',',''))
    except: 
        page['quinella_1'], page['quinella_2'], page['quinella_div'] = 0, 0, 0
    try:
        page['quinella_place_11'] = int(quinella_place[2].split(',')[0])
        page['quinella_place_12'] = int(quinella_place[2].split(',')[1])
        page['quinella_place_21'] = int(quinella_place[4].split(',')[0])
        page['quinella_place_22'] = int(quinella_place[4].split(',')[1])
        page['quinella_place_31'] = quinella_place_31
        page['quinella_place_32'] = quinella_place_32
        page['quinella_place_div_1'] = float(quinella_place[3].replace(',',''))
        page['quinella_place_div_2'] = float(quinella_place[5].replace(',',''))
        page['quinella_place_div_3'] = quinella_place_div_3
    except: 
        page['quinella_place_11'], page['quinella_place_12'] = 0, 0
        page['quinella_place_21'], page['quinella_place_22'] = 0, 0
        page['quinella_place_31'], page['quinella_place_32'] = 0, 0
        page['quinella_place_div_1'], page['quinella_place_div_2'] = 0, 0
        page['quinella_place_div_3'] = 0
        
    # isolate exotic lines
    try:
        page['forecast_1'] = int(forecast[1].split(',')[0])
        page['forecast_2'] = int(forecast[1].split(',')[1])
        page['forecast_div'] = float(forecast[2].replace(',',''))
    except: 
        page['forecast_1'],page['forecast_2'],page['forecast_div'] = 0,0,0
    try:
        page['tierce_1'] = int(tierce[1].split(',')[0])
        page['tierce_2'] = int(tierce[1].split(',')[1])
        page['tierce_3'] = int(tierce[1].split(',')[2])
        page['tierce_div'] = float(tierce[2].replace(',',''))
    except: page['tierce_1'],page['tierce_2'],page['tierce_3'],page['tierce_div'] = 0,0,0,0
    try:
        page['trio_1'] = int(trio[1].split(',')[0])
        page['trio_2'] = int(trio[1].split(',')[1])
        page['trio_3'] = int(trio[1].split(',')[2])
        page['trio_div'] = float(trio[2].replace(',',''))
    except: page['trio_1'],page['trio_2'],page['trio_3'],page['trio_div'] = 0,0,0,0
    try:
        page['first4_1'] = int(first4[2].split(',')[0])
        page['first4_2'] = int(first4[2].split(',')[1])
        page['first4_3'] = int(first4[2].split(',')[2])
        page['first4_4'] = int(first4[2].split(',')[3])
        page['first4_div'] = float(first4[3].replace(',',''))
    except: page['first4_1'],page['first4_2'],page['first4_3'],page['first4_4'],page['first4_div'] = 0,0,0,0,0
    try:
        page['quartet_1'] = int(quartet[1].split(',')[0])
        page['quartet_2'] = int(quartet[1].split(',')[1])
        page['quartet_3'] = int(quartet[1].split(',')[2])
        page['quartet_4'] = int(quartet[1].split(',')[3])
        page['quartet_div'] = float(quartet[2].replace(',',''))
    except: page['quartet_1'],page['quartet_2'],page['quartet_3'],page['quartet_4'],page['quartet_div'] = 0,0,0,0,0
        
    
    print(i, 'div info')
    
    # append extracted race data and output
    races = races.append(page.iloc[:,1:])

0 race info
0 div info
1 race info
1 div info
2 race info
2 div info
3 race info
3 div info
4 race info
4 div info
5 race info
5 div info
6 race info
6 div info
7 race info
7 div info
8 race info
8 div info
9 race info
9 div info
10 race info
10 div info
11 race info
11 div info
12 race info
12 div info
13 race info
13 div info
14 race info
14 div info
15 race info
15 div info
16 race info
16 div info
17 race info
17 div info
18 race info
18 div info
19 race info
19 div info
20 race info
20 div info
21 race info
21 div info
22 race info
22 div info
23 race info
23 div info
24 race info
24 div info
25 race info
25 div info
26 race info
26 div info
27 race info
27 div info
28 race info
28 div info
29 race info
29 div info
30 race info
30 div info
31 race info
31 div info
32 race info
32 div info
33 race info
33 div info
34 race info
34 div info
35 race info
35 div info
36 race info
36 div info
37 race info
37 div info
38 race info
38 div info
39 race info
39 div info
40 race info
40 div 

312 div info
313 race info
313 div info
314 race info
314 div info
315 race info
315 div info
316 race info
316 div info
317 race info
317 div info
318 race info
318 div info
319 race info
319 div info
320 race info
320 div info
321 race info
321 div info
322 race info
322 div info
323 race info
323 div info
324 race info
324 div info
325 race info
325 div info
326 race info
326 div info
327 race info
327 div info
328 race info
328 div info
329 race info
329 div info
330 race info
330 div info
331 race info
331 div info
332 race info
332 div info
333 race info
333 div info
334 race info
334 div info
335 race info
335 div info
336 race info
336 div info
337 race info
337 div info
338 race info
338 div info
339 race info
339 div info
340 race info
340 div info
341 race info
341 div info
342 race info
342 div info
343 race info
343 div info
344 race info
344 div info
345 race info
345 div info
346 race info
346 div info
347 race info
347 div info
348 race info
348 div info
349 race info
3

620 div info
621 race info
621 div info
622 race info
622 div info
623 race info
623 div info
624 race info
624 div info
625 race info
625 div info
626 race info
626 div info
627 race info
627 div info
628 race info
628 div info
629 race info
629 div info
630 race info
630 div info
631 race info
631 div info
632 race info
632 div info
633 race info
633 div info
634 race info
634 div info
635 race info
635 div info
636 race info
636 div info
637 race info
637 div info
638 race info
638 div info
639 race info
639 div info
640 race info
640 div info
641 race info
641 div info
642 race info
642 div info
643 race info
643 div info
644 race info
644 div info
645 race info
645 div info
646 race info
646 div info
647 race info
647 div info
648 race info
648 div info
649 race info
649 div info
650 race info
650 div info
651 race info
651 div info
652 race info
652 div info
653 race info
653 div info
654 race info
654 div info
655 race info
655 div info
656 race info
656 div info
657 race info
6

929 div info
930 race info
930 div info
931 race info
931 div info
932 race info
932 div info
933 race info
933 div info
934 race info
934 div info
935 race info
935 div info
936 race info
936 div info
937 race info
937 div info
938 race info
938 div info
939 race info
939 div info
940 race info
940 div info
941 race info
941 div info
942 race info
942 div info
943 race info
943 div info
944 race info
944 div info
945 race info
945 div info
946 race info
946 div info
947 race info
947 div info
948 race info
948 div info
949 race info
949 div info
950 race info
950 div info
951 race info
951 div info
952 race info
952 div info
953 race info
953 div info
954 race info
954 div info
955 race info
955 div info
956 race info
956 div info
957 race info
957 div info
958 race info
958 div info
959 race info
959 div info
960 race info
960 div info
961 race info
961 div info
962 race info
962 div info
963 race info
963 div info
964 race info
964 div info
965 race info
965 div info
966 race info
9

In [115]:
races = races.reset_index(drop=True)
races['race_id'] = races.reset_index()['index']
races.to_csv('races.csv')
races.head() #[races == 0].info() # check for non-values (0) in payout features

,date,race,number,class,distance,ratings,going,handicap,surface,surface_type,...,first4_2,first4_3,first4_4,first4_div,quartet_1,quartet_2,quartet_3,quartet_4,quartet_div,race_id
0,01/01/2021,1,306,Class 4,1200,(60-40),GOOD,PALM,TURF,B+2,...,8,10,14,19576.0,8,10,14,6,213778.0,0
1,01/01/2021,2,307,Class 4,1400,(60-40),GOOD,POPLAR,TURF,B+2,...,6,7,11,3209.0,6,11,4,7,36762.0,1
2,01/01/2021,3,308,Group Three,1000,,GOOD,THE BAUHINIA SPRINT TROPHY,TURF,B+2,...,0,0,0,0.0,0,0,0,0,0.0,2
3,01/01/2021,4,309,Class 5,1800,(40-0),GOOD,WILLOW,TURF,B+2,...,3,7,14,3678.0,3,14,7,1,107709.0,3
4,01/01/2021,5,310,Class 4,1400,(60-40),GOOD,POPLAR,TURF,B+2,...,9,11,12,1233.0,11,1,9,12,34624.0,4


### Run Data

In [4]:
# define extraction data structures
runs = pd.DataFrame()

# process data in race_tab
for i in range(0,pages.shape[0]):
    
    print(i)
        
    # isolate race runs & split
    text = pages.loc[i,'performance']
    lines = text.splitlines()[2:-1]
    c = 0
    
    # combine lines as new data begins every three lines
    for line in lines: #[:n]:
        
        # initial load
        if c == 0:
            
            # define temp structure and quick values
            page = pd.DataFrame(np.zeros(1))
            page['date'] = pages.loc[i,'date']
            page['race'] = pages.loc[i,'race']
                
            # split lines & iterate counter
            line = line.split()
            b = True
            c += 1

            # isolate horse, jockey, & trainer name
            names, horse, jockey, trainer = '', '', '', ''
            for l in line[2:-4]: names += ' ' + l
            names = pd.Series(names.replace('-',' ').split())
            ih = names.apply(len).idxmin()
            it = names[ih+2:].apply(len).idxmin()
            for l in names[:ih]: horse += ' ' + l
            for l in names[ih:it]: jockey += ' ' + l
            for l in names[it:]: trainer += ' ' + l
                
            # load data to columns
            page['horse'] = horse[:-6]
            page['horse_id'] = horse[-5:-1]
            page['jockey'] = jockey
            page['trainer'] = trainer
            try: 
                page['place'] = int(line[0])
                page['horse_no'] = int(line[1])
                page['weight_adj'] = int(line[-4])
                page['weight_horse_declared'] = int(line[-3])
                page['draw'] = int(line[-2])
            except: b = False
            page['length_behind_winner'] = length_to_float(line[-1])
            
        # load position data
        elif c == 1:
            if b == True: 
                # load data to columns & iterate counter
                line = pd.Series(line.split()).apply(int).tolist() + ['','','','','']
                col = ['pos_1','pos_2','pos_3','pos_4','pos_5','pos_6']
                page[col] = pd.DataFrame(line).T.iloc[:,:6]
                c += 1
            
        # load finishing time and win_odds data
        else:
            # split lines, load data, and reset counter
            line = line.split()
            page['position_finish_time'] = to_float(line[-2])
            page['win_odds'] = float(line[-1])
            c = 0
            
            # append extracted race data
            if b == True: runs = runs.append(page.iloc[:,1:])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:
#runs = pd.read_csv('runs.csv').iloc[:,1:]
runs = runs.reset_index(drop=True)
runs.to_csv('runs.csv')
runs

,date,race,horse,horse_id,jockey,trainer,place,horse_no,weight_adj,weight_horse_declared,draw,length_behind_winner,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,position_finish_time,win_odds
0,01/01/2021,1,MASTER FAY,E162,J Moreira,C Fownes,1,8,126,1185,11,0.00,7,6,1,,,,69.68,1.6
1,01/01/2021,1,TRAVEL GLORY,E170,V Borges,C W Chang,2,10,126,1047,1,2.50,5,4,2,,,,70.09,122.0
2,01/01/2021,1,BRILLIANT,C005,K C Leung,C H Yip,3,14,118,1015,3,3.75,8,9,3,,,,70.29,21.0
3,01/01/2021,1,JOYFUL WIN,D451,T Piccone,P F Yiu,4,6,126,1130,2,4.00,1,1,4,,,,70.31,72.0
4,01/01/2021,1,VIGOR CHAMP,C498,Z Purton,K H Ting,5,12,121,998,13,4.00,13,10,5,,,,70.34,6.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11946,01/09/2019,10,BEAUTY ENERGY,B149,K Teetan,J Moore,10,8,121,1073,8,5.00,2,2,3,10,,,82.38,6.3
11947,01/09/2019,10,PARTY TOGETHER,C210,N Callan,L Ho,11,3,129,1035,11,5.50,9,9,11,11,,,82.47,285.0
11948,01/09/2019,10,CHUNG WAH SPIRIT,A168,T H So,C H Yip,12,2,128,1090,9,7.25,13,14,13,12,,,82.76,199.0
11949,01/09/2019,10,JOLLY CONVERGENCE,V293,L Hewitson,R Gibson,13,12,118,1074,12,9.00,14,13,14,13,,,83.04,175.0


### Horse Data

In [3]:
# define extraction data structures
horse_profiles = pd.DataFrame()

# process data in race_tab
for i in range(0,horses.shape[0]):
    
    #load text and split lines
    text = horses.loc[i,'horseProfile']
    
    try:
        # split lines, isolate, & split words
        lines = text.splitlines()[0:28]
        line_name = lines[0].split()
        line_age_country, line_color_sex, line_sire, line_dam, line_dams_sire = '','','','',''
        for l in lines: 
            if l[0:7] == 'Country': line_age_country = l.split()
            if l[0:6] == 'Colour': line_color_sex = l.split()
            if l[0:4] == 'Sire': line_sire = l.split()
            if l[0:4] == 'Dam ': line_dam = l.split()
            if l[0:4] == 'Dam\'': line_dams_sire = l.split()

        # check if retired
        if line_name[-1] == '(Retired)': 
            line_name = line_name[:-1]
            age = ''
            country = line_age_country[-1]
        else:
            age = int(line_age_country[-1])
            country = line_age_country[-3]

        # concat names
        name, dam, sire, dams_sire = '','','',''
        for l in line_name[:-1]: name += ' ' + l
        for l in line_sire[2:]: sire += ' ' + l
        for l in line_dam[2:]: dam += ' ' + l
        for l in line_dams_sire[3:]: dams_sire += ' ' + l

        # fill horse profile data
        horse = pd.DataFrame(np.zeros(1))
        horse['name'] = name
        horse['horse_id'] = line_name[-1][1:-1]
        horse['age'] = age
        horse['country'] = country
        horse['colour'] = line_color_sex[-1]
        horse['sex'] = line_color_sex[-3]
        horse['sire'] = sire
        horse['dam'] = dam
        horse['dams_sire'] = dams_sire

        # save horse profile
        horse_profiles = horse_profiles.append(horse.iloc[:,1:])
        print('Success:', i)
        
    except:
        horse = pd.DataFrame(np.zeros(1))
        horse_profiles = horse_profiles.append(horse.iloc[:,1:])
        print('Failed:', i)

Success: 0
Success: 1
Success: 2
Success: 3
Success: 4
Success: 5
Success: 6
Success: 7
Success: 8
Success: 9
Success: 10
Success: 11
Success: 12
Success: 13
Success: 14
Success: 15
Success: 16
Success: 17
Success: 18
Success: 19
Success: 20
Success: 21
Success: 22
Success: 23
Success: 24
Success: 25
Failed: 26
Success: 27
Success: 28
Success: 29
Success: 30
Success: 31
Success: 32
Success: 33
Success: 34
Success: 35
Success: 36
Success: 37
Success: 38
Success: 39
Success: 40
Success: 41
Success: 42
Success: 43
Success: 44
Success: 45
Success: 46
Success: 47
Success: 48
Success: 49
Success: 50
Success: 51
Success: 52
Success: 53
Success: 54
Success: 55
Success: 56
Success: 57
Success: 58
Success: 59
Success: 60
Success: 61
Success: 62
Success: 63
Success: 64
Success: 65
Success: 66
Success: 67
Success: 68
Success: 69
Success: 70
Success: 71
Success: 72
Success: 73
Success: 74
Success: 75
Success: 76
Success: 77
Success: 78
Success: 79
Success: 80
Success: 81
Success: 82
Success: 83
Suc

Success: 673
Success: 674
Success: 675
Success: 676
Success: 677
Success: 678
Success: 679
Success: 680
Success: 681
Success: 682
Success: 683
Success: 684
Success: 685
Success: 686
Success: 687
Success: 688
Success: 689
Success: 690
Success: 691
Success: 692
Success: 693
Success: 694
Success: 695
Success: 696
Success: 697
Success: 698
Success: 699
Success: 700
Success: 701
Success: 702
Success: 703
Success: 704
Success: 705
Success: 706
Failed: 707
Success: 708
Success: 709
Success: 710
Success: 711
Success: 712
Success: 713
Success: 714
Success: 715
Success: 716
Success: 717
Success: 718
Success: 719
Success: 720
Success: 721
Success: 722
Success: 723
Success: 724
Success: 725
Success: 726
Failed: 727
Success: 728
Success: 729
Success: 730
Success: 731
Success: 732
Success: 733
Success: 734
Success: 735
Success: 736
Success: 737
Success: 738
Success: 739
Success: 740
Success: 741
Success: 742
Success: 743
Success: 744
Success: 745
Success: 746
Success: 747
Success: 748
Success: 749
S

Success: 1283
Success: 1284
Success: 1285
Success: 1286
Success: 1287
Success: 1288
Success: 1289
Success: 1290
Success: 1291
Success: 1292
Success: 1293
Success: 1294
Success: 1295
Success: 1296
Success: 1297
Success: 1298
Success: 1299
Failed: 1300
Success: 1301
Success: 1302
Success: 1303
Success: 1304
Failed: 1305
Success: 1306
Success: 1307
Success: 1308
Success: 1309
Success: 1310
Success: 1311
Success: 1312
Success: 1313
Success: 1314
Success: 1315
Success: 1316
Success: 1317
Success: 1318
Success: 1319
Success: 1320
Success: 1321
Failed: 1322
Success: 1323
Success: 1324
Success: 1325
Success: 1326
Success: 1327
Success: 1328
Success: 1329
Success: 1330
Success: 1331
Success: 1332
Success: 1333
Success: 1334
Success: 1335
Success: 1336
Success: 1337
Success: 1338
Success: 1339
Success: 1340
Success: 1341
Success: 1342
Success: 1343
Success: 1344
Success: 1345
Success: 1346
Success: 1347
Success: 1348
Success: 1349
Success: 1350
Success: 1351
Success: 1352
Success: 1353
Failed: 1

In [5]:
horse_profiles = horse_profiles.reset_index(drop=True)
horse_profiles.to_csv('horse_profiles.csv')
horse_profiles.head()

,name,horse_id,age,country,colour,sex,sire,dam,dams_sire
0,TRAVEL GLORY,E170,3,NZ,Gelding,Brown,Savabeel,Iguazu's Girl,Redoute's Choice
1,BRILLIANT,C005,,AUS,Gelding,Bay,Encosta de Lago,True Roman,Is It True
2,JOYFUL WIN,D451,3,AUS,Gelding,Bay,Shooting To Win,Miss Kistler,Darci Brahma
3,VIGOR CHAMP,C498,5,AUS,Gelding,Bay,Mossman,Eve Angelene,Catbird
4,CHAMPION PRIDE,B049,7,GB,Gelding,Bay,Invincible Spirit,Loch Jipp,


### Horse History Data

In [13]:
# define extraction data structures
horse_runs = pd.DataFrame()

# process data in race_tab
for i in range(0,horses.shape[0]):
    
    # isolate horse_id
    text = horses.loc[i,'horseProfile']
    print(i)
    
    try:
        # isolate horse_id
        lines = text.splitlines()[0:28]
        line_name = lines[0].split()

        # check if retired
        if line_name[-1] == '(Retired)': line_name = line_name[:-1]
        if line_name[-1] == '(Deregistered)': line_name = line_name[:-1]
        horse_id = line_name[-1][1:-1]

        #load text and split lines
        text = horses.loc[i,'bigborder']
        lines = text.splitlines()[8:]

        for line in lines:
            if line[0] != ' ':

                # split line
                line = line.split()
                b = True

                # isolate jockey & trainer name, positions
                names, jockey, trainer, positions = '', '', '', ''
                for l in line[12:-3]: names += ' ' + l
                names = pd.Series(names.replace('-',' ').replace('/',' ').replace('SH',' ').replace(' N',' ').replace(' HD',' ').split())
                ih = names.apply(len).idxmin() # start of trainers name
                it = names[ih+2:].apply(len).idxmin() # length of trainers name
                num = []
                for n in names[it:]:
                    try: num += [int(n)]
                    except: num += [n]
                ip = 0 # length of jockeys name
                for n in num:
                    try:
                        x = float(n)
                        break
                    except: ip += 1
                for l in names[ih:it]: trainer += ' ' + l
                for l in names[it:it+ip]: jockey += ' ' + l
                for l in line[12+it+ip+3:-3]: positions += ' ' + l

                # fill horse run data
                try:
                    run = pd.DataFrame(np.zeros(1))
                    run['horse_id'] = horse_id
                    run['race_idx'] = int(line[0])
                    run['place'] = int(line[1])
                    run['date'] = line[2]
                    run['race_track'] = line[3]
                    run['surface'] = line[5]
                    run['surface_type'] = line[7][1:-1]
                    run['distance'] = int(line[8])
                    run['g_race'] = line[9]
                    run['class'] = line[10]
                    run['draw'] = int(line[11])
                    run['rating'] = int(line[12])

                    run['trainer'] = trainer
                    run['jockey'] = jockey
                    run['length_behind_winner'] = length_to_float(line[12+it+ip])
                    run['win_odds'] = float(line[12+it+ip+1])
                    run['weight_actual'] = int(line[12+it+ip+2])

                    positions = pd.Series(positions.split()).apply(int).tolist() + ['','','','','']
                    col = ['pos_1','pos_2','pos_3','pos_4','pos_5','pos_6']
                    run[col] = pd.DataFrame(positions).T.iloc[:,:6]

                    run['position_finish_time'] = to_float(line[-3])
                    run['weight_horse_declared'] = int(line[-2])
                    run['gear'] = line[-1]
                except: b = False

                # save horse profile
                if b == True: horse_runs = horse_runs.append(run.iloc[:,1:])

    except:
        horse = pd.DataFrame(np.zeros(1))
        horse_runs = horse_runs.append(run.iloc[:,1:])
        print('Failed:', i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
Failed: 113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
27

C:\Users\shanc\Anaconda3\lib\site-packages\ipykernel_launcher.py:74: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199
1200
1201
1202
1203
1204
1205
1206
1207
1208
1209
1210
1211
1212
1213
1214
1215
1216
1217
1218
1219
1220
1221
1222
1223
1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
Failed: 1233
1234
1235
1236
1237
1238
1239
Failed: 1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
1262
1263
1264
1265
1266
1267
1268
1269
1270
1271
1272
1273
Failed: 1273
1

In [15]:
horse_runs = horse_runs.reset_index(drop=True)
horse_runs.to_csv('horse_runs.csv')
horse_runs

,horse_id,race_idx,place,date,race_track,surface,surface_type,distance,g_race,class,...,weight_actual,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,position_finish_time,weight_horse_declared,gear
0,E170,306,2,01/01/21,ST,Turf,B+2,1200,G,4,...,126.0,5,4,2,,,,70.09,1047.0,--
1,C005,306,3,01/01/2021,ST,Turf,B+2,1200,G,4,...,118.0,8,9,3,,,,70.29,1015.0,TT
2,C005,198,9,18/11/2020,HV,Turf,C,1000,G,4,...,123.0,12,11,9,,,,58.01,1020.0,H-/TT
3,C005,810,5,12/07/2020,ST,Turf,A,1200,GF,4,...,123.0,12,9,5,,,,69.58,1000.0,H/TT
4,C005,783,8,01/07/2020,ST,Turf,C,1200,GF,4,...,124.0,12,11,8,,,,70.17,1003.0,H/TT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20906,B417,71,13,06/10/2019,ST,Turf,B+2,1200,G,4,...,119.0,5,4,13,,,,73.69,1147.0,TT
20907,B417,415,9,13/02/2019,HV,Turf,B,1000,G,4,...,124.0,6,7,9,,,,58.28,1147.0,TT1
20908,B417,323,11,09/01/2019,HV,Turf,A,1000,G,4,...,126.0,5,2,11,,,,59.39,1149.0,--
20909,B417,256,10,16/12/2018,ST,Turf,C+3,1000,G,4,...,125.0,6,3,10,,,,57.88,1147.0,--
